# netCDF4

* netCDF4是一种常见数据存储格式，通常以`.nc`、`.p`作为文件后缀（也可以像wrfout那样没有后缀）。
* netCDF4一般包含数据本身及其元数据（坐标、单位、物理意义等），即文件具有自描述性，能够方便使用者查看数据的具体情况。

## 文件读取

* `ncview`可以快速预览文件。
* `netCDF4`的`Dataset`可以读取文件以绘图或数据处理。  
  `login2`、`node`节点在白天（时段不确定）可能无法读取`netCDF4`文件，该问题很早就已知但暂无解决办法。  
  `mgt`全天可读取，但是不允许在`mgt`运行脚本，因此只建议在有急用的情况下**偷偷地**先在`mgt`读取出来并保存为NumPy的`.npy`文件，再去其他节点作后续处理。
* 这里以《大气数值模式及应用》第四次作业（理想试验）文件为例。

In [1]:
from pathlib import Path

from netCDF4 import Dataset

filepath = Path(
    "/public/home/XiaAnRen/data3/vscode/data/course/Atmo-modeling/exp04/wrfout",
)
ncfile = Dataset(filepath)

## 查看文件信息

* 直接`print`文件可以看到运行WRF时的配置信息（`DX`、`DY`、投影、参数化方案等），以及文件所包含的维度（`dimensions`）与变量（`variables`）。

In [2]:
ncfile

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    TITLE:  OUTPUT FROM WRF V4.6.0 MODEL
    START_DATE: 0001-01-01_00:00:00
    SIMULATION_START_DATE: 0001-01-01_00:00:00
    WEST-EAST_GRID_DIMENSION: 42
    SOUTH-NORTH_GRID_DIMENSION: 42
    BOTTOM-TOP_GRID_DIMENSION: 41
    DX: 2000.0
    DY: 2000.0
    AERCU_OPT: 0
    AERCU_FCT: 1.0
    IDEAL_CASE: 2
    DIFF_6TH_SLOPEOPT: 0
    AUTO_LEVELS_OPT: 2
    DIFF_6TH_THRESH: 0.1
    DZBOT: 50.0
    DZSTRETCH_S: 1.3
    DZSTRETCH_U: 1.1
    SKEBS_ON: 0
    USE_Q_DIABATIC: 0
    GRIDTYPE: C
    DIFF_OPT: 2
    KM_OPT: 2
    DAMP_OPT: 2
    DAMPCOEF: 0.003
    KHDIF: 500.0
    KVDIF: 500.0
    MP_PHYSICS: 28
    RA_LW_PHYSICS: 0
    RA_SW_PHYSICS: 0
    SF_SFCLAY_PHYSICS: 0
    SF_SURFACE_PHYSICS: 0
    BL_PBL_PHYSICS: 0
    CU_PHYSICS: 0
    SF_LAKE_PHYSICS: 0
    SURFACE_INPUT_SOURCE: 3
    SST_UPDATE: 0
    GHG_INPUT: 1
    GRID_FDDA: 0
    GFDDA_INTERVAL_M: 0
    GFDDA_END_H: 0
    GRID_SFDDA: 0
   

* 通过`Dataset.ncattrs()`可以看到前述的“运行WRF时的配置信息”有哪些。

In [3]:
ncfile.ncattrs()

['TITLE',
 'START_DATE',
 'SIMULATION_START_DATE',
 'WEST-EAST_GRID_DIMENSION',
 'SOUTH-NORTH_GRID_DIMENSION',
 'BOTTOM-TOP_GRID_DIMENSION',
 'DX',
 'DY',
 'AERCU_OPT',
 'AERCU_FCT',
 'IDEAL_CASE',
 'DIFF_6TH_SLOPEOPT',
 'AUTO_LEVELS_OPT',
 'DIFF_6TH_THRESH',
 'DZBOT',
 'DZSTRETCH_S',
 'DZSTRETCH_U',
 'SKEBS_ON',
 'USE_Q_DIABATIC',
 'GRIDTYPE',
 'DIFF_OPT',
 'KM_OPT',
 'DAMP_OPT',
 'DAMPCOEF',
 'KHDIF',
 'KVDIF',
 'MP_PHYSICS',
 'RA_LW_PHYSICS',
 'RA_SW_PHYSICS',
 'SF_SFCLAY_PHYSICS',
 'SF_SURFACE_PHYSICS',
 'BL_PBL_PHYSICS',
 'CU_PHYSICS',
 'SF_LAKE_PHYSICS',
 'SURFACE_INPUT_SOURCE',
 'SST_UPDATE',
 'GHG_INPUT',
 'GRID_FDDA',
 'GFDDA_INTERVAL_M',
 'GFDDA_END_H',
 'GRID_SFDDA',
 'SGFDDA_INTERVAL_M',
 'SGFDDA_END_H',
 'HYPSOMETRIC_OPT',
 'USE_THETA_M',
 'GWD_OPT',
 'SF_URBAN_PHYSICS',
 'SF_SURFACE_MOSAIC',
 'SF_OCEAN_PHYSICS',
 'SLUCM_DISTRIBUTED_DRAG',
 'DISTRIBUTED_AHE_OPT',
 'SHCU_PHYSICS',
 'MFSHCONV',
 'FEEDBACK',
 'SMOOTH_OPTION',
 'SWRAD_SCAT',
 'W_DAMPING',
 'RADT',
 'BLDT',
 'C

* 通过`Dataset.getncattr(attr)`可以看到前述配置信息具体的值。
* 例如这里给出该wrfout的水平分辨率为$2 \ \mathrm{km}$。

In [4]:
ncfile.getncattr("DX")

2000.0

## 读取变量的值

### 基于netCDF4

* `Dataset.variables`以字典形式存储了变量名及其对应的具体数值。

In [5]:
type(ncfile.variables)

dict

* `keys()`对应所有的变量名。
* `values()`对应具体值。

In [6]:
ncfile.variables.keys()

dict_keys(['Times', 'XLAT', 'XLONG', 'LU_INDEX', 'ZNU', 'ZNW', 'ZS', 'DZS', 'VAR_SSO', 'BATHYMETRY_FLAG', 'U', 'V', 'W', 'PH', 'PHB', 'T', 'THM', 'HFX_FORCE', 'LH_FORCE', 'TSK_FORCE', 'HFX_FORCE_TEND', 'LH_FORCE_TEND', 'TSK_FORCE_TEND', 'MU', 'MUB', 'NEST_POS', 'P', 'PB', 'FNM', 'FNP', 'RDNW', 'RDN', 'DNW', 'DN', 'CFN', 'CFN1', 'THIS_IS_AN_IDEAL_RUN', 'P_HYD', 'Q2', 'T2', 'TH2', 'PSFC', 'U10', 'V10', 'RDX', 'RDY', 'AREA2D', 'DX2D', 'RESM', 'ZETATOP', 'CF1', 'CF2', 'CF3', 'ITIMESTEP', 'XTIME', 'QVAPOR', 'QCLOUD', 'QRAIN', 'QICE', 'QSNOW', 'QGRAUP', 'QNWFA2D', 'QNIFA2D', 'QNICE', 'QNRAIN', 'QNCLOUD', 'QNWFA', 'QNIFA', 'QNBCA', 'SHDMAX', 'SHDMIN', 'SHDAVG', 'SNOALB', 'TSLB', 'SMOIS', 'SH2O', 'SEAICE', 'XICEM', 'SFROFF', 'UDROFF', 'IVGTYP', 'ISLTYP', 'VEGFRA', 'GRDFLX', 'ACGRDFLX', 'ACSNOM', 'SNOW', 'SNOWH', 'CANWAT', 'SSTSK', 'WATER_DEPTH', 'COSZEN', 'LAI', 'VAR', 'O3_GFS_DU', 'MAPFAC_M', 'MAPFAC_U', 'MAPFAC_V', 'MAPFAC_MX', 'MAPFAC_MY', 'MAPFAC_UX', 'MAPFAC_UY', 'MAPFAC_VX', 'MF_VX_INV',

* 使用变量名进行索引即可得到对应的元数据。
* 这里以云水混合比（QCLOUD）为例。
  * `Time, bottom_top, south_north, west_east`表示数据是四维的（时间、高度、南北、东西）。
  * `description: Cloud water mixing ratio`交代了物理意义是云水混合比。
  * `units: kg kg-1`说明单位为$\mathrm{kg \ kg^{-1}}$。
  * `coordinates: XLONG XLAT XTIME`指时间、南北、东西的具体数值保存在了`XLONG`（经度）`XLAT`（纬度）`XTIME`（时间）三个变量内。
  * `current shape = (7, 40, 41, 41)`对应前述的“四维”，即在时间维有7个节点，在垂直方向上分了40层，水平的东西南北各分了41个区域。

In [7]:
ncfile.variables["QCLOUD"]

<class 'netCDF4._netCDF4.Variable'>
float32 QCLOUD(Time, bottom_top, south_north, west_east)
    FieldType: 104
    MemoryOrder: XYZ
    description: Cloud water mixing ratio
    units: kg kg-1
    stagger: 
    coordinates: XLONG XLAT XTIME
unlimited dimensions: Time
current shape = (7, 40, 41, 41)
filling on, default _FillValue of 9.969209968386869e+36 used

* 使用变量名得到元数据后可以继续进行索引得到多维数组。

In [8]:
data = ncfile.variables["QCLOUD"][:]
type(data), data.shape, data

(numpy.ma.core.MaskedArray,
 (7, 40, 41, 41),
 masked_array(
   data=[[[[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           ...,
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]],
 
          [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           ...,
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]],
 
          [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           ...,
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]],
 
          ...,
 
          [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., .

* 支持索引即也支持切片。
* 这里所得到的是第3时刻、第20垂直高度层的数据（从0开始计数）。

In [9]:
data = ncfile.variables["QCLOUD"][3, 20]
type(data), data.shape, data

(numpy.ma.core.MaskedArray,
 (41, 41),
 masked_array(
   data=[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
   mask=False,
   fill_value=1e+20,
   dtype=float32))

### 基于WRF-Python

* 另一种常见的是使用WRF-Python的`getvar`读取数据。
  * 这样可以读取一些wrfout本身没有的诊断量，例如`z`（高度，单位：$\mathrm{m}$）。  
    `getvar`会自动计算这些量的值。  
    更多变量详见<https://wrf-python.readthedocs.io/en/latest/user_api/generated/wrf.getvar.html>
  * 不同`.nc`文件里的变量名有可能不一样。  
    例如“地形高度”会记为`HGT`或`ter`。  
    `getvar`不会受此影响，不需要针对不同`.nc`的命名方式专门修改脚本。
* `getvar(wrfin, varname)`将返回包含元数据的对象。  
  通过`getvar(wrfin, varname).to_numpy()`或`getvar(wrfin, varname).values`可以得到对应的多维数组。

In [10]:
from wrf import getvar

data = getvar(ncfile, "QCLOUD").to_numpy()
type(data), data.shape, data

(numpy.ndarray,
 (40, 41, 41),
 array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         

* 上一代码块得到的是三维数组而非四维。
* 原因在于`getvar`有一个指定时间索引的参数`timeidx`。
* 默认情况下`timeidx=0`，对应第0时刻的数组。
* 如果想要读取所有时刻的数组，就需要额外导入`ALL_TIMES`并设置`timeidx=ALL_TIMES`。

In [11]:
from wrf import ALL_TIMES

data = getvar(ncfile, "QCLOUD", timeidx=ALL_TIMES).to_numpy()
type(data), data.shape, data

(numpy.ndarray,
 (7, 40, 41, 41),
 array([[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0